In [1]:
import pandas as pd
import numpy as np

In [2]:
fightersTable = pd.read_csv('/home/chris/anaconda3/raw_fighter_details.csv')  



def getYearofBirth(row):
    if "," in str(row):
        return int (str(row)[row.index(",")+1:].strip())
    
    return float("nan")

def HeightFeetToCm(feet):
    feet = feet.replace('"',"").replace("'","")
    return int(feet[0])*30.48 + int(feet[2:])*2.54

def ReachInchesToCm(inches):
    return int( inches[:-1] )

def setMedian(attribute):
    print(attribute)
    
def setSwitchIfNa(x):
    if x!="Orthodox" and x!="Southpaw" and x!="Switch":
        return "Switch"
    else:
        return x
    
def returnValueIfFound(row):
    if len(row)==0:
        return float(0)
    else:
        return row.values[0]
    
    

    

In [3]:


fightersTable['DOB'] = fightersTable['DOB'].apply( lambda x : getYearofBirth(x) )    #keep only the year of birth

fightersTable = fightersTable[ fightersTable['DOB'].notna() ]
fightersTable = fightersTable[ fightersTable['Reach'].notna() ]
fightersTable = fightersTable[ fightersTable['Height'].notna() ]

fightersTable['Height'] = fightersTable['Height'].apply( lambda x : HeightFeetToCm(x))
fightersTable['Reach'] = fightersTable['Reach'].apply( lambda x : ReachInchesToCm(x))
fightersTable['Stance'] = fightersTable['Stance'].apply( lambda x : setSwitchIfNa(str(x).strip()))

fightersTable.to_csv("/home/chris/anaconda3/UFC FINAL FIGHTER DATA.csv" , index=False)

In [4]:
rawTable = pd.read_csv('/home/chris/anaconda3/UFC DATA.csv')  
columnNames = rawTable.columns
columnNames = [col.replace(" ","") for col in columnNames]
rawTable.columns = columnNames

rawTable['isWomen'] = [ int("Women" in row) for row in rawTable['Class']]    #create isWomen column


def classToNum(x,dictionary):
    for key in dictionary.keys():     
        if  key in x:
            return key
        
    
weights_dict = {
    "Women's Strawweight": 1,                                               #turn the "Class" to numeric
    "Women's Flyweight": 2,
    "Women's Bantamweight": 3,
    "Women's Featherweight": 4,
    "Flyweight" : 5,
    "Bantamweight": 6,
    "Featherweight": 7,
    "Lightweight": 8,
    "Welterweight": 9,
    "Middleweight": 10,
    "Light Heavyweight": 11,
    "Heavyweight": 12,      

}

def fillMissingHeight(row,column):        
    if row[column+'A'] ==0:
        sub = rawTable[rawTable.Class== row.Class]
        sub = sub[sub[column+'A'] !=0]
        row[column+'A'] = sub[column+'A'].mean()
    if row[column+'B']==0:
        sub = rawTable[rawTable.Class== row.Class]
        sub = sub[sub[column+'A'] !=0]
        row[column+'B']= sub[column+'A'].mean()
    return row


rawTable['Class'] = rawTable['Class'].apply( lambda x : classToNum(x,weights_dict) )
rawTable = rawTable[ rawTable['Class'].notna() ]
rawTable['HeightA'] = [ returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['Height']) for row in rawTable['FighterA']]      #create Height columns
rawTable['HeightB'] = [ returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['Height']) for row in rawTable['FighterB']]      

rawTable = rawTable.apply( lambda row: fillMissingHeight(row,'Height'), axis=1)

rawTable['ReachA'] = [ returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['Reach']) for row in rawTable['FighterA']]      #create Reach columns
rawTable['ReachB'] = [ returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['Reach']) for row in rawTable['FighterB']]      
rawTable = rawTable.apply( lambda row: fillMissingHeight(row,'Reach'), axis=1)
rawTable['AgeA'] = [ 2020-returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['DOB']) for row in rawTable['FighterA']]      #create Reach columns
rawTable['AgeB'] = [ 2020-returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['DOB']) for row in rawTable['FighterB']]      
rawTable = rawTable.apply( lambda row: fillMissingHeight(row,'Age'), axis=1)

rawTable['StanceA'] = [ returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['Stance']) for row in rawTable['FighterA']]      #create Reach columns
rawTable['StanceB'] = [ returnValueIfFound(fightersTable[ fightersTable['fighter_name']==str(row)]['Stance']) for row in rawTable['FighterB']]      

rawTable['Method'] = [ method.replace("Could Not Continue","Decision - Unanimous") for method in rawTable['Method'] ]
rawTable['ScheduledRounds'] = [ int(rounds) for rounds in rawTable['ScheduledRounds']  ]

for x in ["A","B"]:
    rawTable['Sig.str.' +x] = [int(row[:row.index("/")]) for row in rawTable['Sig.str.' +x]  ]
    rawTable['Sig.str.%' +x] =  [ int(row[:-1])/100 for row in rawTable['Sig.str.%' +x]  ]
    rawTable['Totalstr.'+x] = [ int(row[:row.index("/")]) for row in rawTable['Totalstr.'+x]  ]
    rawTable['Td'+x] = [int(row[:row.index("/")]) for row in rawTable['Td'+x]   ]
    rawTable['Td%'+x] =  [ int(row[:-1])/100 for row in rawTable['Td%'+x]  ]

rawTable['isTitle'] = [ int('Title' in fight) for fight in rawTable['Class']]
rawTable['Class'] = rawTable['Class'].apply( lambda x : classToNum(x,weights_dict) )
rawTable = rawTable[ rawTable['Class'].notna() ]

rawTable = rawTable[ rawTable.ScheduledRounds >2 ]
rawTable = rawTable[ rawTable.Method != "DQ" ]

rawTable.to_csv("/home/chris/anaconda3/UFC FINAL FIGHT DATA.csv", index=False)


In [5]:
rawTable = pd.read_csv('/home/chris/anaconda3/UFC FINAL FIGHT DATA.csv')  

pos = 0
def countFights(row):
    global pos
    pos +=1
    sub = rawTable[pos:]
    fights = len(sub[sub['FighterA']==row]) + len(sub[sub['FighterB']==row])
    return fights
    

rawTable['num_fightsA'] = [countFights(row) for row in rawTable['FighterA'] ]
pos = 0
rawTable['num_fightsB'] = [countFights(row) for row in rawTable['FighterB'] ]

def countRounds(row):
    global pos
    pos +=1        
    sub = rawTable[pos:] 
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    return subA.FinishRound.sum() + subB.FinishRound.sum()

def countWins(row):
    global pos
    pos +=1
    sub = rawTable[pos:]
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    wins = subA[subA['winnerA']==1].count()[0] + subB[subB['winnerA']==0].count()[0]
    return wins

def calcWinPerc(row,symbol):
    global pos
    pos +=1
    wholeRow = rawTable.iloc[pos-1]
    fights = wholeRow['num_fights'+symbol]
    wins = wholeRow['num_wins'+symbol]
    if fights==0:
        return 0
    return float(wins/fights)

def calcPerRoundAtt(row ,symbol, column):
    global pos
    pos +=1
    wholeRow = rawTable.iloc[pos-1]
    rounds = wholeRow['num_rounds'+symbol]

    sub = rawTable[pos:] 
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    attacks = subA[column+"A"].sum() + subB[column+"B"].sum()
    if rounds==0:
        return 0
    return float(attacks/rounds)
    
def calcPercAtt(row ,symbol, column):
    global pos
    pos +=1
    wholeRow = rawTable.iloc[pos-1]
    fights = wholeRow['num_fights'+symbol]   
    
    
    sub = rawTable[pos:] 
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    attacks = subA[column+"A"].sum() + subB[column+"B"].sum()
    if fights==0:
        return 0
    return float(attacks/fights)

def calcPercReceived(row ,symbol, column):
    global pos
    pos +=1
    wholeRow = rawTable.iloc[pos-1]
    fights = wholeRow['num_fights'+symbol]   
    
    
    sub = rawTable[pos:] 
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    attacks = subA[column+"B"].sum() + subB[column+"A"].sum()
    if fights==0:
        return 0
    return float(attacks/fights)
    
    
def calcPerRoundReceived(row ,symbol, column):
    global pos
    pos +=1
    wholeRow = rawTable.iloc[pos-1]
    rounds = wholeRow['num_rounds'+symbol]
    
    sub = rawTable[pos:] 
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    attacks = subA[column+"B"].sum() + subB[column+"A"].sum()
    if rounds==0:
        return 0
    return float(attacks/rounds)
    
    
def calcPercReceived(row ,symbol, column):
    global pos
    pos +=1
    wholeRow = rawTable.iloc[pos-1]
    fights = wholeRow['num_fights'+symbol]   
      
    sub = rawTable[pos:] 
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    attacks = subA[column+"B"].sum() + subB[column+"A"].sum()
    if fights==0:
        return 0
    return float(attacks/fights)
    
    
def calcFinish(row , column):   
    global pos
    pos +=1
    sub = rawTable[pos:]
    sub = sub[sub.Method.str.contains(column)]
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    finishes = subA[subA.winnerA ==1].count()[0] + subB[subB.winnerA ==0].count()[0]
    return finishes
    
def calcBeenFinished(row , column):     
    global pos
    pos +=1
    sub = rawTable[pos:]
    sub = sub[sub.Method.str.contains(column)]
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    finishes = subA[subA.winnerA ==0].count()[0] + subB[subB.winnerA ==1].count()[0]
    return finishes

def calcTitleFights(row ):     
    global pos
    pos +=1
    sub = rawTable[pos:]
    subA = sub[sub['FighterA']==row]
    subB = sub[sub['FighterB']==row]
    fights = subA[subA.isTitle ==1].count()[0] + subB[subB.isTitle ==1].count()[0]
    return fights


def calcStreakOfWins(row ):     
    global pos
    pos +=1
    sub = rawTable[pos:]
    sub = sub[ sub['FighterA'].str.join(sub['FighterB'].str).contains(row) ]
    
    streak =0
    for index, r in sub.iterrows():
        if (r.FighterA == row and r.winnerA==1) or (r.FighterB == row and r.winnerA==0):
            streak+=1
        else:
            break
            
    return streak



In [ ]:
for x in ["A","B"]:
    pos=0
    rawTable['num_wins'+x] = [ countWins(row) for row in rawTable['Fighter'+x]  ]    
    pos=0
    rawTable['num_rounds'+x] = [ countRounds(row) for row in rawTable['Fighter'+x]  ]    
    pos=0
    rawTable['Sig.str.perRound'+x] = [ calcPerRoundAtt(row,x,"Sig.str.") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['TDperRound'+x] = [ calcPerRoundAtt(row,x,"Td") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['Sig.str.perRoundreceived'+x] = [ calcPerRoundReceived(row,x,"Sig.str.") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['TDperRoundreceived'+x] = [ calcPerRoundReceived(row,x,"Td") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['Sig.str.%avg'+x] = [ calcPercAtt(row,x,"Sig.str.%") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['TD%avg'+x] = [ calcPercAtt(row,x,"Td%") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['Sig.str.def%avg'+x] = [ calcPercReceived(row,x,"Sig.str.%") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['TDdef%avg'+x] = [ calcPercReceived(row,x,"Td%") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['TotalstrperRound'+x] = [ calcPerRoundAtt(row,x,"Totalstr.") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['KDperRound'+x] = [ calcPerRoundAtt(row,x,"KD") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['Sub.attperRound'+x] = [ calcPerRoundAtt(row,x,"Sub.att") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['PassperRound'+x] = [ calcPerRoundAtt(row,x,"Pass") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['RevperRound'+x] = [ calcPerRoundAtt(row,x,"Rev.") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['KOs'+x] = [ calcFinish(row,"KO") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['Subs'+x] = [ calcFinish(row,"Sub") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['beenKO'+x] = [ calcBeenFinished(row,"KO") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['beenSub'+x] = [ calcBeenFinished(row,"Sub") for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['num_title_fights'+x] = [ calcTitleFights(row) for row in rawTable['Fighter'+x]  ]
    pos=0
    rawTable['winStreak'+x] = [ calcStreakOfWins(row) for row in rawTable['Fighter'+x]  ]
    
    rawTable['winPerc'+x] = rawTable.apply(lambda row: row['num_wins'+x]/row['num_fights'+x] if row['num_fights'+x]!=0 else 0, axis=1)
    rawTable['num_losses'+x] = rawTable.apply(lambda row:  row['num_fights'+x]-row['num_wins'+x] , axis=1)
    rawTable['KOPerc'+x] = rawTable.apply(lambda row: row['KOs'+x]/row['num_fights'+x] if row['num_fights'+x]!=0 else 0, axis=1)    
    rawTable['SubPerc'+x] = rawTable.apply(lambda row: row['Subs'+x]/row['num_fights'+x] if row['num_fights'+x]!=0 else 0, axis=1)
    rawTable['beenKOPerc'+x] = rawTable.apply(lambda row: row['beenKO'+x]/row['num_fights'+x] if row['num_fights'+x]!=0 else 0, axis=1)    
    rawTable['beenSubPerc'+x] = rawTable.apply(lambda row: row['beenSub'+x]/row['num_fights'+x] if row['num_fights'+x]!=0 else 0, axis=1)

rawTable['StanceA'] = rawTable['StanceA'].apply( lambda x : setSwitchIfNa(str(x).strip()))
rawTable['StanceB'] = rawTable['StanceB'].apply( lambda x : setSwitchIfNa(str(x).strip()))
rawTable.to_csv("/home/chris/anaconda3/UFC FINAL FIGHT DATA.csv", index=False)
rawTable = rawTable.drop(columns=[ 'KDA', 'Sig.str.A', 'Sig.str.%A', 'Totalstr.A', 'TdA', 'Td%A', 'Sub.attA', 'PassA', 'Rev.A', 'KDB', 'Sig.str.B', 'Sig.str.%B','Totalstr.B', 'TdB', 'Td%B', 'Sub.attB', 'PassB', 'Rev.B', 'Method', 'FinishRound'])
rawTable.to_csv("/home/chris/anaconda3/UFC FINAL DATA.csv", index=False)

In [ ]:
rawTable.columns